# Water Use Estimator Code
---

### Library Imports
---

In [21]:
import requests
import pandas as pd
import time
import datetime
from IPython.display import clear_output

### Data and Cleaning
---

Load in my county lat and longitude data.

In [3]:
counties = pd.read_csv('./data/ohio-county-boundaries.csv')
counties.head()

,Geo Point,Geo Shape,STATEFP,COUNTYFP,COUNTYNS,GEOID,NAME,NAMELSAD,STUSAB,LSAD,...,CSAFP,CBSAFP,METDIVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,STATE_NAME,COUNTYFP NOZERO
0,"40.7715207081, -84.1057794323","{""type"": ""Polygon"", ""coordinates"": [[[-84.3971...",39,3,1074015,39003,Allen,Allen County,OH,6,...,338.0,30620.0,NaN,A,1042479889,11259523,40.771627,-84.106103,Ohio,3
1,"41.9103521337, -81.250939242","{""type"": ""Polygon"", ""coordinates"": [[[-81.4886...",39,85,1074055,39085,Lake,Lake County,OH,6,...,184.0,17460.0,NaN,A,593807218,1942301625,41.924116,-81.392643,Ohio,85
2,"40.7684277475, -80.7771865597","{""type"": ""Polygon"", ""coordinates"": [[[-81.0867...",39,29,1074027,39029,Columbiana,Columbiana County,OH,6,...,566.0,41400.0,NaN,A,1377658481,7277032,40.770073,-80.778455,Ohio,29
3,"41.6350590585, -81.7001959208","{""type"": ""Polygon"", ""coordinates"": [[[-81.9702...",39,35,1074030,39035,Cuyahoga,Cuyahoga County,OH,6,...,184.0,17460.0,NaN,A,1184093716,2041810673,41.760392,-81.724217,Ohio,35
4,"39.6203734727, -81.8526618984","{""type"": ""Polygon"", ""coordinates"": [[[-82.0319...",39,115,1074070,39115,Morgan,Morgan County,OH,6,...,NaN,NaN,NaN,A,1078538278,13868572,39.624946,-81.861697,Ohio,115


Remove all of the unneeded information.

In [4]:
counties = counties[['NAME', 'INTPTLAT', 'INTPTLON']]
counties.to_csv('./data/counties.csv')

Do a quick `.info()` to make sure we don't have any null values or incorrect dtypes

In [5]:
counties.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88 entries, 0 to 87
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   NAME      88 non-null     object 
 1   INTPTLAT  88 non-null     float64
 2   INTPTLON  88 non-null     float64
dtypes: float64(2), object(1)
memory usage: 2.2+ KB


Create a list of counties for use in our Streamlit selectbox.

In [6]:
county_list = counties['NAME'].tolist()
county_list

['Allen',
 'Lake',
 'Columbiana',
 'Cuyahoga',
 'Morgan',
 'Harrison',
 'Putnam',
 'Muskingum',
 'Tuscarawas',
 'Wyandot',
 'Clark',
 'Butler',
 'Pike',
 'Ottawa',
 'Hamilton',
 'Coshocton',
 'Fulton',
 'Van Wert',
 'Washington',
 'Wood',
 'Fairfield',
 'Preble',
 'Logan',
 'Lawrence',
 'Pickaway',
 'Jackson',
 'Athens',
 'Stark',
 'Auglaize',
 'Brown',
 'Belmont',
 'Medina',
 'Paulding',
 'Darke',
 'Noble',
 'Union',
 'Delaware',
 'Mahoning',
 'Carroll',
 'Meigs',
 'Henry',
 'Champaign',
 'Vinton',
 'Lorain',
 'Franklin',
 'Hancock',
 'Crawford',
 'Highland',
 'Huron',
 'Adams',
 'Guernsey',
 'Morrow',
 'Erie',
 'Clermont',
 'Licking',
 'Hardin',
 'Williams',
 'Summit',
 'Geauga',
 'Scioto',
 'Defiance',
 'Seneca',
 'Mercer',
 'Richland',
 'Perry',
 'Gallia',
 'Madison',
 'Trumbull',
 'Hocking',
 'Miami',
 'Ashland',
 'Shelby',
 'Portage',
 'Lucas',
 'Sandusky',
 'Montgomery',
 'Ross',
 'Jefferson',
 'Fayette',
 'Wayne',
 'Clinton',
 'Greene',
 'Marion',
 'Monroe',
 'Ashtabula',
 'Hol

Write a function that will:
1. Check to see what the date is and whether we need to roll over into the next month
2. Compile a list of URLs to scrape for the specified location using lat and long coordinates and the user specified date
3. Pull the desired weather information from the json from our API request
4. Compile our data into a dataframe to store the information for use in our calculations
4. Run the calculations to get our final water estimate numbers

**NOTE**: Streamlit is Pythonic in nature but certain bits of code didn't work verbatim.  You can view the exact Streamlit code in my project repo in the 'water_app.py' file which is located in the'app_pages' folder.

In [31]:
# Define some reference variables that will be used within our function.

years = list(range(2009, 2021))
months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
days = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
month_day_dict = {months[i]: days[i] for i in range(len(months))}

def water_estimate(month_str, day_int, county, size_of_garden):
    '''
    This function serves to do a few things:
    1. Check to see what the date is and whether we need to roll over into the next month
    2. Compile a list of URLs to scrape for the specified location using lat and long coordinates and the user specified date
    3. Pull the desired weather information from the json from our API request
    4. Compile our data into a dataframe to store the information for use in our calculations
    4. Run the calculations to get our final water estimate numbers
    '''
    # first determine if we even need to roll over to the next month
    county_coords = counties.loc[counties['NAME'] == county, ['INTPTLAT', 'INTPTLON']]
    lat = str(round(float(county_test['INTPTLAT']), 2))
    long = str(round(float(county_test['INTPTLON']), 2))
    
    if day_int+7 > month_day_dict[month_str]:
        
        # if true, find the next month by finding its index in the dictionary, adding one, and casting as a string (also add a 0 in front to work properply with WWO's API formatting)
        next_month = '0' + str(list(month_day_dict.keys()).index(month_str)+2)
        
        # determine how many days into the next month we must go
        next_month_day = month_day_dict[month_str] - day_int
        next_month_day = 7 - next_month_day
        
        # define our new start and end variables
        start_date = month_str + '-' + str(day_int)
        end_date = next_month + '-' +str(next_month_day)

    else:  
        
        # otherwise we just use the start and end dates
        start_date = month_str + '-' + str(day_int)
        end_date = month_str + '-' + str(day_int+7)
    
    # Next we need to create the URLs for our API requests by concatenating all of our strings together within the specificed date range as well as going back to previous years.
    urls = []
    for year in years:
        
        url = 'http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key=ee035e8c8a26404e99b183309212110' 
        url = url + '&format=json'
        url = url + '&q=' + str(lat) + ',' + str(long)
        url = url + '&tp=24'
        url = url + '&date=' + str(year) + '-' + start_date
        url = url + '&enddate=' + str(year) + '-' + end_date
        
        urls.append(url)   

    # Compile the data from our requests into a list of dictionaries
    data = []   
    for i, url in enumerate(urls):
        time.sleep(1)
        res = requests.get(url)
        data.append(res.json())
     
    # Create an empty list to append all of our desired weather features and create columns for our dataframe
    list_of_cols = ['date', 'maxtempF', 'mintempF', 'precipInches']
    list_of_records = []
    for i in range(len(data)):
        for num in range(0, 7):
            list_of_records.append([data[i]['data']['weather'][num]['date'], 
                int(data[i]['data']['weather'][num]['maxtempF']), 
                int(data[i]['data']['weather'][num]['mintempF']), 
                float(data[i]['data']['weather'][num]['hourly'][0]['precipInches'])])
        
    df = pd.DataFrame(list_of_records, columns=list_of_cols)
    
    # Find the averages from our dataframe
    average_temp = round((df['maxtempF'].mean() + df['mintempF'].mean()) / 2, 2)
    average_precip = round(df['precipInches'].mean(), 2)
    size_of_garden = int(size_of_garden)
    
    # Create conditional statements for whether the temp was above or below 60 F
    if average_temp > 60:
        # If the temp was above 60 we have to find the difference and divide that by 10.  For every 10 degrees above 60 we can expect to need to use roughly .5 inches more water.
        # We then add that number to the normal 1 inch of water normally needed per week.
        # Then I multiplied that amount by 1.56 gallons per whatever our inches of water is and multiplied that by the square footage of the garden to get total gallons of water needed.
        diff = 80 - average_temp
        water_amt = round(((diff / 10) * .5) + (1 - average_precip), 2)
        gallons = round((water_amt * 1.56) * size_of_garden, 2)
        print(f"The average temperature for the specified time period is: {average_temp} F")
        print()
        print(f"The average rainfall for the specified time period is: {average_precip} inches")
        print()
        print(f"Using the above historical averages, we anticipate you will need to water your plants about {water_amt} inches per sq/ft or roughly {gallons} gallons of water for your entire garden")
        
    else:
        water_amt = 1 - average_precip
        gallons = round((water_amt * 1.56) * size_of_garden, 2)
        print(f"The average temperature for the specified time period is: {average_temp} F")
        print()
        print(f"The average rainfall for the specified time period is: {average_precip} inches")
        print()
        print(f"Using the above historical averages, we anticipate you will need to water your plants about {water_amt} inches per sq/ft or roughly {gallons} gallons of water for you entire garden")

In [32]:
water_estimate('08', 12, 'Fairfield', 100)

The average temperature for the specified time period is: 70.17 F

The average rainfall for the specified time period is: 0.17 inches

Using the above historical averages, we anticipate you will need to water your plants about 1.32 inches per sq/ft or roughly 205.92 gallons of water for your entire garden
